In [19]:
import pandas as pd
import os
import copy
import re
from tqdm import tqdm

In [20]:
#CHECK MADE ONLY ON PSALMS (it is longer thus it may contains all interesting cases)
#'Unlocked Literal Bible__.pdf', #OK
#Easy-to-Read Version__American  #OK
#Young's Literal Translation__archaic British #OK
#JPS TaNaKH 1917__American #OK
#Noah Webster Bible__archaic American #OK
#-King James (Authorized) Version__archaic British #OK



In [21]:
def read_csv(folder_idx,file_idx):
    folder_='csv_folders/'
    #get folders
    in_folders_=[x for x in os.listdir(folder_)]
    in_folders_
    #print('#FOLDERS:\n'+ '\n-'.join(in_folders_)+'\n____________')
    #get files of folders
    in_files_=sorted([x for x in os.listdir(folder_+in_folders_[folder_idx])], key=lambda x:int(x.split('-')[0]))
    in_files_
    #print('#FILES:\n'+'\n-'.join(in_files_)+'\n_____________')
    #read file
    df=pd.read_csv(folder_+in_folders_[folder_idx]+'/'+in_files_[file_idx])
    #print('FOLDER: '+in_folders_[folder_idx]+' - FILE: '+in_files_[file_idx])
    return df, [in_folders_[folder_idx], in_files_[file_idx].replace('.csv','')]

df, folder_file=read_csv(8,23) #fold 4



In [22]:
def split_verses(df_):
    df_temp=copy.deepcopy(df_)
    for v_,verse in enumerate(df_temp['Verses']):
        match_l=re.findall('([\s][\d]+[\s])',verse)
        new_verses=[]
        if match_l:  
            for i,match_ in enumerate(match_l):
                temp_l=[x for x in verse.split(match_l[i]) if re.findall('[^-]$',x)]
                for j,each in enumerate(temp_l):
                    if j>0:
                        update_verse= str(int(re.sub('\)|\]|\}','', temp_l[0].split(':')[1].split(' ')[0]))+1)
                        temp_l[j]=temp_l[0].split(':')[0]+':'+ update_verse +' '+ each
                #print(temp_l)
            for i, new_row in enumerate(temp_l):
                if i>0:
                    df_temp.loc[df_temp.index[v_]+.5]=[df_temp['Chapter'][v_],new_row]
    return df_temp.sort_index().reset_index(drop=True)

In [23]:
def find_spourious_verses(df_):
    df_temp= copy.deepcopy(df_)
    target_list=['year','th[\s]','st[\s]','people','day','cubit','palm',
                 'span','gallon','handbreadth','ell','descendant', 'man','men','women','woman',
                'quart','beka', 'gerah','name','Israelite','shekels','‡ men', 'night','feet', 'trained',
                'soldier','piece','male','female','cow','bull','camel','gram','ounce','shekel',
                'pound','liter','mina','kilogram', 'liter','double','bath','ephah','quart',
                'omer','meter','mile','stadia','stadium','centimeter','inch','Year','elder','months','inch',
                'province']
    footer_target_list=['Chron.',]
    verses_col=df_temp['Verses']
    pattern_=r'[\d]+:([\d]+[\s]?)('+"|".join(target_list)+r')'
    #aggiungi al verso precedente tutte le righe che iniziano per [\d]+[\s][targe_list elements]
    v_starting_with_one=[]
    v_others=[]
    
    idx_to_drop=[]
    for i,verse_ in enumerate(verses_col):
        footer_status=any([x in verse_ for x in footer_target_list ])
        #print(footer_status)
        if footer_status:
                #print('ALT', i,verse_)
                for j_idx in df_temp.iloc[i:,:].index:
                    #print(int(df_temp.iloc[int(j_idx),1].split(':')[0])-1)
                    try:
                        cap_=int(df_temp.iloc[int(j_idx),1].split(':')[0])
                    except Exception as e:
                        #print(i,j,e)
                        continue #se non inizia per numero allora skippa
            
                    cap_-=1
                    verse_=df_temp.iloc[int(j_idx),1].split(':')[1]
                    new_str=str(cap_)+':'+verse_
                    df_temp.iloc[int(j_idx),1]=new_str   
                
        anomaly_match=re.findall(pattern_,verse_)
        if len(anomaly_match)>=1: 
            if re.match('(^1[\D])', ''.join(verse_.split(':')[1:])):#se inizia con uno
                #print('AGGIORNA CAPITOLI',i,'-',verse_)
                #update following chaps
                for j_idx in df_temp.iloc[i:,:].index:
                    #print(int(df_temp.iloc[int(j_idx),1].split(':')[0])-1)
                    try:
                        cap_=int(df_temp.iloc[int(j_idx),1].split(':')[0])
                    except Exception as e:
                        #print(i,j,e)
                        continue #se non inizia per numero allora skippa
                    cap_-=1
                    verse_=df_temp.iloc[int(j_idx),1].split(':')[1]
                    new_str=str(cap_)+':'+verse_
                    df_temp.iloc[int(j_idx),1]=new_str
            else:# se non inizia per 1
                if i>0:
                    print(anomaly_match[-1])
                    anomaly_match_res=' '.join([x for x in anomaly_match[-1]])
                    print(anomaly_match_res)
                    temp_str= ' '.join(verse_.split(anomaly_match_res)[-1].split(':')[1:])
                    print(temp_str)
                    df_temp.iloc[df_temp.index[i]-1,1]=df_temp.iloc[df_temp.index[i]-1,1]+' '+temp_str
                    idx_to_drop.append(df_temp.index[i])
                    #df_temp.drop(df_temp.index[i],axis=0,inplace=True)
                    #df_temp.reset_index(drop=True)
            pass#print('TROVATO',i,'-',verse_)
        ##if re.findall(r"(:1[\s]?\w+[\s]?[\d]+?[\s]–)", verse_):
        ##    print('AGGIORNA 2')
        ##    for j_idx in df_temp.iloc[i:,:].index:
        ##            print(int(df_temp.iloc[int(j_idx),1].split(':')[0])-1)
        ##            try:
        ##                cap_=int(df_temp.iloc[int(j_idx),1].split(':')[0])
        ##            except Exception as e:
        ##                print(i,j,e)
        ##                continue #se non inizia per numero allora skippa
        ##            cap_-=1
        ##            verse_=df_temp.iloc[int(j_idx),1].split(':')[1]
        ##            new_str=str(cap_)+':'+verse_
        ##            df_temp.iloc[int(j_idx),1]=new_str
        ##    if df_temp['Verses'][0].strip().startswith('0'):
        ##        for j_idx in df_temp.iloc[i:,:].index:
        ##            print(int(df_temp.iloc[int(j_idx),1].split(':')[0])-1)
        ##            try:
        ##                cap_=int(df_temp.iloc[int(j_idx),1].split(':')[0])
        ##            except Exception as e:
        ##                print(i,j,e)
        ##                continue #se non inizia per numero allora skippa
        ##            cap_+=1
        ##            verse_=df_temp.iloc[int(j_idx),1].split(':')[1]
        ##            new_str=str(cap_)+':'+verse_
        ##            df_temp.iloc[int(j_idx),1]=new_str
    print(idx_to_drop)
    df_temp.drop(idx_to_drop,axis=0,inplace=True)
    return df_temp


#new_df.tail(50)

In [24]:
def clean_and_save(df_, folder_file_idxs):
    new_df=split_verses(df_)   
    new_df=find_spourious_verses(new_df)
    
    final_df=pd.DataFrame()
    final_df['Edition']=[folder_file_idxs[0] for x in new_df['Verses']]
    final_df['Book']=[x for x in new_df['Chapter']]
    final_df['Chapter']=[x.split(':')[0] for x in new_df['Verses']]
    final_df['Verse']=[''.join(x.split(':')[1:]) for x in new_df['Verses']]
    
    os.makedirs('clean_csv_folders/'+folder_file_idxs[0],exist_ok=True)
    file_name=folder_file_idxs[1].split('-')[0]+'-'+folder_file_idxs[1].split(';')[-1]
    
    final_df.to_csv('clean_csv_folders/'+folder_file_idxs[0]+'/'+file_name+'.csv',index=False,header=True)


In [25]:
folder_file

['Unlocked Literal Bible__', '66-Unlocked Literal Bible__;Revelation']

In [26]:
for i,folder_ in enumerate(os.listdir('csv_folders/')):
    print(i, folder_)
    for j, file_ in enumerate(tqdm(os.listdir('csv_folders/'+folder_+'/'))):
        df, folder_file=read_csv(i,j) #fold 4
        clean_and_save(df, folder_file)  
        


0 Unlocked Literal Bible__


  2%|█▎                                                                                 | 1/66 [00:00<00:07,  9.02it/s]

('130 ', 'year')
130  year
130 years, he became the father of a son in his own likeness, after his image, and he called his name Seth. 
('930 ', 'year')
930  year
930 years, and then he died. 
('105 ', 'year')
105  year
105 years, he became the father of Enosh. 
('807 ', 'year')
807  year
807 years and became the father of more sons and daughters. 
('912 ', 'year')
912  year
912 years, and then he died. 
('815 ', 'year')
815  year
815 years. He became the father of more sons and daughters. 
('905 ', 'year')
905  year
905 years, and then he died. 
('840 ', 'year')
840  year
840 years. He became the father of more sons and daughters. 
('910 ', 'year')
910  year
910 years, and then he died. 
('830 ', 'year')
830  year
830 years. He became the father of more sons and daughters. 
('895 ', 'year')
895  year
895 years, and then he died. 
('162 ', 'year')
162  year
162 years, he became the father of Enoch. 
('962 ', 'year')
962  year
962 years, and then he died. 
('365 ', 'year')
365  year
365

  5%|███▊                                                                               | 3/66 [00:00<00:04, 13.20it/s]

[]
('500 ', 'men')
500  men
500 men from the tribe of Reuben. 
('300 ', 'men')
300  men
300 men from the tribe of Simeon. 
('650 ', 'men')
650  men
650 men from the tribe of Gad. 
('600 ', 'men')
600  men
600 men from the tribe of Judah. 
('400 ', 'men')
400  men
400 men from the tribe of Issachar.
('400 ', 'men')
400  men
400 men from the tribe of Zebulun. 
('500 ', 'men')
500  men
500 men from the tribe of Ephraim. 
('200 ', 'men')
200  men
200 men from the tribe of Manasseh. 
('400 ', 'men')
400  men
400 men from the tribe of Benjamin. 
('500 ', 'men')
500  men
500 men from the tribe of Asher. 
('550 ', 'men')
550  men
550 men. 
('400 ', 'men')
400  men
400 men. 
('600 ', 'male')
600  male
600 males have been counted aged one month old and older to take care of the things that belong to Yahweh. 
('200 ', 'male')
200  male
200 males have been counted aged one month old and older. 
('273 ', 'men')
273  men
273 men. 
('365 ', 'shekels')
365  shekels
365 shekels, weighing with the sheke

...

[]
[]


 40%|█████████████████████████████████▌                                                  | 2/5 [00:00<00:00, 14.10it/s]

[]
[]


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 15.03it/s]

[]


In [30]:
#CONCAT
for i,folder_ in enumerate(os.listdir('clean_csv_folders/')):
    print(i, folder_)
    l_dfs=[pd.read_csv('clean_csv_folders/'+folder_+'/'+x) for x in os.listdir('clean_csv_folders/'+folder_+'/')]
    concat_df=pd.concat(l_dfs)
    concat_df=concat_df[~concat_df['Verse'].isna()].reset_index(drop=True)
    concat_df.to_csv('clean_csv_folders/'+folder_+'/'+'CONCAT'+'_'+folder_+'.csv',index=False,header=True)   

0 Unlocked Literal Bible__
1 Easy-to-Read Version__American
2 Young's Literal Translation__archaic British
3 JPS TaNaKH 1917__American
4 Revised Version with Apocrypha (1895)__archaic British
5 Noah Webster Bible__archaic American
6 King James (Authorized) Version__archaic British
7 Open English Bible (Commonwealth Spelling)__
8 Open English Bible (U. S. spelling)__
9 Targum Onkelos Etheridge__


#result
clean_and_save(df, folder_file)

In [ ]:
re.e("one|two", "three", 'one two two')

In [ ]:
' '.join([x for x in ('asd','fds')])

In [ ]:
final_df

NameError: name 'x' is not defined

In [ ]:
###
coords_=['for','and','but','nor','or','yet','so']

In [50]:
temp_subords=""" after
 although
 as
 as if
 as long as
 as much as
 as soon as
 as though
 because
 before
 even
 even if
 even though
 if
 if only
 if when
 if then
 inasmuch
 in order that
 just as
 lest
 now
 now since
 now that
 now when
 once
 provided
 provided that
 rather than
 since
 so that
 supposing
 than
 that
 though
 til
 unless
 until
 when
 whenever
 where
 whereas
 where if
 wherever
 whether
 which
 while
 who
 whoever
 why
""".split('\n')

In [51]:
temp_subords=[' '.join(x.split(' ')[1:]).strip() for x in temp_subords]
temp_subords

['after',
 'although',
 'as',
 'as if',
 'as long as',
 'as much as',
 'as soon as',
 'as though',
 'because',
 'before',
 'even',
 'even if',
 'even though',
 'if',
 'if only',
 'if when',
 'if then',
 'inasmuch',
 'in order that',
 'just as',
 'lest',
 'now',
 'now since',
 'now that',
 'now when',
 'once',
 'provided',
 'provided that',
 'rather than',
 'since',
 'so that',
 'supposing',
 'than',
 'that',
 'though',
 'til',
 'unless',
 'until',
 'when',
 'whenever',
 'where',
 'whereas',
 'where if',
 'wherever',
 'whether',
 'which',
 'while',
 'who',
 'whoever',
 'why',
 '']

In [52]:
[temp_subords.append(x.strip().lower()) for x in """After
If only 
Unless
Although 
In order that 
Until
As 
Now that 
When
As if 
Once 
Whenever
As long as 
Provided 
Where
As though
Rather than
Whereas
Because
Since
Wherever
Before 
So that 
Whether
Even if 
That 
While
Even though 
Though 
Within
If 
Without 
Besides""".split('\n')]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [53]:
temp_subords

['after',
 'although',
 'as',
 'as if',
 'as long as',
 'as much as',
 'as soon as',
 'as though',
 'because',
 'before',
 'even',
 'even if',
 'even though',
 'if',
 'if only',
 'if when',
 'if then',
 'inasmuch',
 'in order that',
 'just as',
 'lest',
 'now',
 'now since',
 'now that',
 'now when',
 'once',
 'provided',
 'provided that',
 'rather than',
 'since',
 'so that',
 'supposing',
 'than',
 'that',
 'though',
 'til',
 'unless',
 'until',
 'when',
 'whenever',
 'where',
 'whereas',
 'where if',
 'wherever',
 'whether',
 'which',
 'while',
 'who',
 'whoever',
 'why',
 '',
 'after',
 'if only',
 'unless',
 'although',
 'in order that',
 'until',
 'as',
 'now that',
 'when',
 'as if',
 'once',
 'whenever',
 'as long as',
 'provided',
 'where',
 'as though',
 'rather than',
 'whereas',
 'because',
 'since',
 'wherever',
 'before',
 'so that',
 'whether',
 'even if',
 'that',
 'while',
 'even though',
 'though',
 'within',
 'if',
 'without',
 'besides']

In [54]:
[temp_subords.append(x.strip()) for x in """as
because
in order
so that
now that
since
although
as
as though
even
though
just as
though
whereas
while
even if
if
in case
provided
unless
where
wherever
after
as
as soon as
as long as
before
once
still
till
until
when
whenever
while
as
as though
though
whereas
while
although
""".split('\n')]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [57]:
list(set(temp_subords))

['',
 'as though',
 'supposing',
 'whereas',
 'till',
 'which',
 'in order that',
 'even if',
 'as long as',
 'now that',
 'since',
 'before',
 'where if',
 'so that',
 'because',
 'as soon as',
 'til',
 'while',
 'in order',
 'inasmuch',
 'after',
 'whoever',
 'once',
 'as',
 'now',
 'besides',
 'as if',
 'if',
 'that',
 'if only',
 'still',
 'whenever',
 'if then',
 'wherever',
 'though',
 'whether',
 'rather than',
 'unless',
 'although',
 'now since',
 'in case',
 'until',
 'as much as',
 'why',
 'where',
 'within',
 'if when',
 'even though',
 'provided',
 'when',
 'than',
 'who',
 'just as',
 'provided that',
 'without',
 'even',
 'lest',
 'now when']